### Final project Part 3: Viz for others (Ian Chapman)
# How much might renewable electricity generation reduce our carbon imprint?

In [1]:
import pandas as pd
import numpy as np
import bqplot
import ipywidgets

## I. WA total emissions per sector (bar chart)

In [2]:
ghg = pd.read_csv('WA_GHG_Reporting_Multi-Year_Dataset(county_mod).csv',
                  na_values = {'2016 total emissions (MTCO2e)': ''})

ghg = ghg.dropna(axis=0, subset=['2016 total emissions (MTCO2e)'])

ghg


# I have modified the ghg dataset as follows:
# In the "County" column, "NA" is changed to "(Statewide)".
# This allows data not ascribed to a given county to be included in the heatmap below.

# Citation: Method for removing nulls in selected column adapted from: 
# https://stackoverflow.com/questions/49291740/delete-rows-if-there-are-null-values-in-a-specific-column-in-pandas-dataframe
# (viewed 4/13/19)

,Source,Sector,Subsector,City,County,Local Air Authority,2012 total emissions (MTCO2e),2012 biogenic carbon dioxide (MTCO2e),2013 total emissions (MTCO2e),2013 biogenic carbon dioxide (MTCO2e),2014 total emissions (MTCO2e),2014 biogenic carbon dioxide (MTCO2e),2015 total emissions (MTCO2e),2015 biogenic carbon dioxide (MTCO2e),2016 total emissions (MTCO2e),2016 biogenic carbon dioxide (MTCO2e),2017 total emissions (MTCO2e),2017 biogenic carbon dioxide (MTCO2e)
0,Agrium Kennewick Fertilizer Operations (KFO) -...,Chemicals,Nitric Acid Production,Kennewick,Benton,Benton Clean Air Agency,146926.0,0.0,154497.0,0.0,132249.0,0.0,155888.0,0.0,151371.0,0.0,144290.0,0.0
1,Air Liquide - Anacortes,Chemicals,Hydrogen Production,Anacortes,Skagit,Northwest Clean Air Agency,63356.0,0.0,58995.0,0.0,64110.0,0.0,64413.0,0.0,60209.0,0.0,63461.0,0.0
2,Alcoa Intalco Works - Ferndale,Metals,Aluminum Production,Ferndale,Whatcom,Ecology: Industrial Section,1146835.0,0.0,1234637.0,0.0,1326684.0,0.0,1195786.0,0.0,1261364.0,0.0,1091665.0,0.0
3,Alcoa Wenatchee Works - Malaga,Metals,Aluminum Production,Malaga,Chelan,Ecology: Industrial Section,306333.0,0.0,318542.0,0.0,354692.0,0.0,331207.0,0.0,898.0,0.0,0.0,0.0
4,Alon Asphalt Company - Seattle,Petroleum and Natural Gas Systems,Other Petroleum and Natural Gas Systems,Seattle,King,Puget Sound Clean Air Agency,15138.0,0.0,14336.0,0.0,16004.0,0.0,13688.0,0.0,14096.0,0.0,14818.0,0.0
5,Ardagh Glass Inc. - Seattle,Minerals,Glass Production,Seattle,King,Puget Sound Clean Air Agency,76257.0,0.0,80745.0,0.0,78044.0,0.0,76674.0,0.0,77845.0,0.0,75338.0,0.0
6,Ascensus Specialties LLC - Elma,Chemicals,Other Chemicals,Elma,Grays Harbor,Olympic Region Clean Air Agency,16809.0,0.0,17966.0,0.0,21231.0,0.0,17600.0,0.0,20802.0,0.0,21310.0,0.0
7,Ash Grove Cement Company - Seattle,Minerals,Cement Production,Seattle,King,Puget Sound Clean Air Agency,305298.0,0.0,354808.0,0.0,522982.0,0.0,495030.0,0.0,383836.0,0.0,355513.0,0.0
8,Avista Corporation - WA State DOE Reporting - ...,Petroleum and Natural Gas Systems,Natural Gas Local Distribution Companies,Spokane,Spokane,Spokane Regional Clean Air Agency,20992.0,0.0,16127.0,0.0,16420.0,0.0,22858.0,0.0,21120.0,0.0,23757.0,0.0
9,Basic American Foods - Moses Lake,Food Production,Potato Products,Moses Lake,Grant,Ecology: Eastern Regional Office,28205.0,0.0,28312.0,0.0,28982.0,0.0,31063.0,0.0,28977.0,0.0,30576.0,0.0


In [3]:
x = ghg['Sector']
y = ghg['2016 total emissions (MTCO2e)']

xnames = x.unique()
ynames = y.unique()

for i,xn in enumerate(xnames):
    mask = (x == xn)
    ynames[i] = y[mask].sum()

x_sc = bqplot.OrdinalScale()
y_sc = bqplot.LinearScale()

x_ax = bqplot.Axis(scale = x_sc, 
                    label = 'Sector',
                    label_offset = '60px',
                    tick_rotate = 70,
                    tick_style = {'font-size':'12px'},
                    offset = {'scale':x_sc, 'value':'60px'})
y_ax = bqplot.Axis(scale = y_sc, 
                    orientation = 'vertical', 
                    side = 'left',
                    label = 'Emissions (MT CO2e)',
                    label_offset = '50px')

sect_bar = bqplot.Bars(x = xnames,
                     y = ynames,
                     color_mode = 'element',
                     scales = {'x': x_sc, 'y': y_sc},
                    opacities = [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5],
                    interactions = {'click': 'select'},
                    anchor_style = {'fill':'red'}, 
                    selected_style = {'fill':'red','opacity': 0.5},
                    unselected_style = {'opacity': 1.0})


fig_sect = bqplot.Figure(marks = [sect_bar],
                         axes = [x_ax, y_ax],
                        fig_margin = {'top':60, 'bottom':120, 'left':70, 'right':40},
                        title = "WA greenhouse gas emissions by sector, 2016")

#  I have set "opacities" in bqplot.Bars to 0.5 for each of the 13 bars
#  (apparently you need to do them individually), to make the intruding tick labels visible. 
# But this has not worked: the bars remain fully opaque.
# Setting "opacity" to 0.5 under "unselected style" does make the selected bar translucent.

### Interaction: Breakdown of sectors into subsectors (bar chart)

In [4]:
x2 = ghg['Subsector'].values
y2 = ghg['2016 total emissions (MTCO2e)'].values

x2_sc = bqplot.OrdinalScale() 
y2_sc = bqplot.LinearScale()

x2_ax = bqplot.Axis(scale = x2_sc,
                    label = 'Subsector',
                    label_offset = '70px',
                    tick_values = x2,
                    tick_rotate = 45,
                    tick_style = {'font-size':'12px'})
y2_ax = bqplot.Axis(scale = y2_sc,
                    label = 'Emissions (MT CO2e)',
                    label_offset = '50px',
                    orientation = 'vertical',
                    side = 'left')

i = 0
mask = (x.values == xnames[i])
subsect = x2[mask]
emis2 = y2[mask]

emis2 = emis2[~pd.isnull(subsect)]
subsect = subsect[~pd.isnull(subsect)]

subsectu = np.unique(subsect)
emis2u = [emis2[subsect == subsect[i]].sum() for i in range(len(subsectu)) ]

subsect_bar = bqplot.Bars(x = subsectu,
                          y = emis2u,
                          color_mode = 'element',
                          scales = {'x': x2_sc, 'y': y2_sc})

fig_subsect = bqplot.Figure(marks = [subsect_bar], 
                            axes = [x2_ax, y2_ax],
                            fig_margin = {'top':60, 'bottom':120, 'left':70, 'right':60},
                            title = "Emissions by subsector")


In [5]:
emis_tot = ghg['2016 total emissions (MTCO2e)'].sum()
emis_tot

63948300.0

In [6]:
mySelectedLabel1a = ipywidgets.Label()
mySelectedLabel1b = ipywidgets.Label()

def get_data_value(change):
    if change['owner'].selected is not None:
        i = change['owner'].selected[0]
        mask = (x.values == xnames[i])
        subsect = x2[mask]
        emis2 = y2[mask]
        emis2 = emis2[~pd.isnull(subsect)]
        subsect = subsect[~pd.isnull(subsect)]
        subsectu = np.unique(subsect)
        emis2 = [emis2[subsect == subsectu[b]].sum() for b in range(len(subsectu)) ]
        emis2 = np.array(emis2)
        v = emis2.sum()
        pct = v/emis_tot*100
        mySelectedLabel1a.value = 'Sector GHG emissions = ' + str(v)
        mySelectedLabel1b.value = 'Percentage of total = ' + str(round(pct,1))
        subsect_bar.x = subsectu
        subsect_bar.y = emis2

sect_bar.observe(get_data_value, 'selected')

fig_sect.layout.max_width = '500px'
fig_sect.layout.max_height= '500px'
fig_subsect.layout.max_width='300px'
fig_subsect.layout.max_height='400px'



## II. Comparison: US GHG sector breakdown

In [7]:
epa_raw = pd.read_csv('EPA sectors 1990-2017.csv')

exclude = ['Total', 'U.S. territories']

epa = epa_raw[~epa_raw['Economic Sector'].isin(exclude)]

# Citation: Pandas dataframe filter method adapted from:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html
# (viewed 4/18/19)

In [8]:
xE = epa['Economic Sector']
yE = epa['2016']

xEnames = xE.unique()
yEnames = yE.unique()

for i,xEn in enumerate(xEnames):
    maskE = (xE == xEn)
    yEnames[i] = yE[maskE].sum()

xE_sc = bqplot.OrdinalScale()
yE_sc = bqplot.LinearScale()

xE_ax = bqplot.Axis(scale = xE_sc, 
                    label = 'Sector',
                    label_offset = '60px',
                    tick_rotate = 45,
#                     tick_style = {'font-size':'10px', 'tick_offset':'100px','text_anchor':'top'})
                    tick_style = {'font-size':'12px'},
                    offset = {'scale':x_sc, 'value':'60px'})
yE_ax = bqplot.Axis(scale = yE_sc, 
                    orientation = 'vertical', 
                    side = 'left',
                    label = 'Emissions (MT CO2e)',
                    label_offset = '50px')

epa_bar = bqplot.Bars(x = xEnames,
                     y = yEnames,
                     color_mode = 'element',
                     scales = {'x': xE_sc, 'y': yE_sc},
                    interactions = {'click': 'select'},
                    anchor_style = {'fill':'red'}, 
                    selected_style = {'fill':'red','opacity': 0.5},
                    unselected_style = {'opacity': 1.0})


fig_epa = bqplot.Figure(marks = [epa_bar],
                         axes = [xE_ax, yE_ax],
                        fig_margin = {'top':60, 'bottom':120, 'left':70, 'right':60},
                        title = "US greenhouse gas emissions by sector, 2016")


fig_epa.layout.max_width = '500px'
fig_epa.layout.max_height= '500px'


### Interaction: Selected Label

In [9]:
emisE_tot = epa['2016'].sum()
emisE_tot

mySelectedLabelEa = ipywidgets.Label()
mySelectedLabelEb = ipywidgets.Label()


def get_data_valueE(change):
    if change['owner'].selected is not None:
        i = change['owner'].selected[0]
        maskE = (xE.values == xEnames[i])
        sectE = xE[maskE]
        emisE = yE[maskE]
        emisE = emisE[~pd.isnull(sectE)]
        sectE = sectE[~pd.isnull(sectE)]
        sectEu = np.unique(sectE)
        emisEu = [emisE[sectE == sectEu[b]].sum() for b in range(len(sectEu)) ]
        emisEu = np.array(emisEu)
        vE = emisEu.sum()
        pctE = vE/emisE_tot*100
        mySelectedLabelEa.value = 'US sector GHG emissions = ' + str(vE)
        mySelectedLabelEb.value = 'Percentage of total = ' + str(round(pctE,1))
   
epa_bar.observe(get_data_valueE, 'selected')


## III. WA economic sector, county, emissions (heat map)

In [10]:
x3 = ghg['Sector']
y3 = ghg['County']
z3 = ghg['2016 total emissions (MTCO2e)']

x3names = x3.unique()
y3names = y3.unique()
z3names = np.zeros([len(x3names),len(y3names)])

for i,x3n in enumerate(x3names):
    for j, y3n in enumerate(y3names):
        mask3 = (x3 == x3n) & (y3 == y3n)
        z3names[i,j] = z3[mask3].sum()

In [11]:
col_sc = bqplot.ColorScale(scheme="RdPu")
x3_sc = bqplot.OrdinalScale()
y3_sc = bqplot.OrdinalScale()

c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')

x3_ax = bqplot.Axis(scale = x3_sc,
                    label='County',
                    label_offset = '50px',
                    tick_rotate=90,
                    tick_style = {'font-size':'12px'},
                    offset = {'scale':x3_sc, 'value':'50'})
y3_ax = bqplot.Axis(scale = y3_sc, 
                    orientation = 'vertical', 
                    label = 'Sector',
                    label_offset = '120px',
                    tick_style = {'font-size':'12px'})

heat_map = bqplot.GridHeatMap(color = np.log10(z3names),
                              row = x3names, 
                              column = y3names,
                              scales = {'color': col_sc,
                                        'row': y3_sc,
                                        'column': x3_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})

fig_hm = bqplot.Figure(marks = [heat_map],
                       axes = [c_ax, y3_ax, x3_ax], 
                       fig_margin = dict(top=60, bottom=80, left=200, right=50),
                       title = "WA greenhouse gas emissions by sector and county, 2016")

fig_hm.layout.width = '900px'
fig_hm.layout.height= '500px'



C:\Users\Ian\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log10


### Interaction: Emissions per subsector per county (bar chart)

In [12]:
x4 = ghg['Subsector'].values
y4 = ghg['2016 total emissions (MTCO2e)'].values

x4_sc = bqplot.OrdinalScale() 
y4_sc = bqplot.LinearScale()

x4_ax = bqplot.Axis(scale=x4_sc,
                    label='Subsector',
                    label_offset = '40px',
                    tick_rotate=10,
                    tick_style={'font-size':'12px'})
y4_ax = bqplot.Axis(scale=y4_sc,
                    label='Emissions (MT CO2e)',
                    label_offset = '50px',
                    orientation='vertical') 


In [13]:
i,j = 0,0
mask3 = (x3.values == x3names[i]) & (y3.values == y3names[j])
subsect4 = x4[mask3]
emis4 = y4[mask3]

emis4 = emis4[~pd.isnull(subsect4)]
subsect4 = subsect4[~pd.isnull(subsect4)]

subsect4u = np.unique(subsect4)
emis4u = [emis4[subsect4 == subsect4[i]].sum() for i in range(len(subsect4u)) ]

In [14]:
bar4 = bqplot.Bars(x = subsect4u,
                  y = emis4u,
                  color_mode = 'element',
                  scales = {'x': x4_sc, 'y': y4_sc})

fig_bar4 = bqplot.Figure(marks = [bar4],
                        axes = [x4_ax, y4_ax],
                        fig_margin = {'top':60, 'bottom':120, 'left':70, 'right':50},
                        title = 'County emissions by subsector')

fig_bar4.layout.max_width='300px'
fig_bar4.layout.max_height='400px'


In [15]:
mySelectedLabel2 = ipywidgets.Label()

def get_data_value3(change):
    i,j = change['owner'].selected[0]
    mask3 = (x3.values == x3names[i]) & (y3.values == y3names[j])
    subsect4 = x4[mask3]
    emis4 = y4[mask3]
    emis4 = emis4[~pd.isnull(subsect4)]
    subsect4 = subsect4[~pd.isnull(subsect4)]
    subsect4u = np.unique(subsect4)
    emis4u = [emis4[subsect4 == subsect4u[b]].sum() for b in range(len(subsect4u)) ]
    emis4u = np.array(emis4u)
    v = emis4u.sum(),
    mySelectedLabel2.value = 'Sector GHG emissions for county = ' + str(v)
    bar4.x = subsect4u
    bar4.y = emis4u
    
heat_map.observe(get_data_value3, 'selected')


## Greenhouse gas generation: reading the data

Nationally, power generation through fossil fuels is a major contributor to global warming. What impact might we expect from the wide scale adoption of renewable power sources? We look at the data from Washington, a state in which renewables - largely hydroelectricity - already account for 90% of electricity production. 

All data is from the year 2016, the most recent for which comparative datasets are available. Emissions are calculated in metric tons (MT) of carbon dioxide equivalents (CO2e), a measure which converts all greenhouse gases to an equivalent volume of carbon dioxide, to provide a basis for direct comparison.

### 1. Breakdown by sector of Washington state's greenhouse gas emissions 
We begin by looking at Washington state's greenhouse gas emission profile. The visualization shows that electricity generation (power plants) accounts for almost 16% of the state's greenhouse gas production. This is a distant second to transportation fuel supply (51%); combined with the related industry of refining (10%), oil production and use production account for 61% of the state's emissions.

How much variation is there within each sector? Click on the bar for a given sector to see the contributions of individual subsectors. For example, the "Power plants" breakdown shows that coal plants still account for the largest proportion of fossil-fuel power generation, though natural gas plants are catching up.

Data is provided by the Washington Department of Ecology (available at: https://data.wa.gov/Natural-Resources-Environment/WA-GHG-Reporting-Multi-Year-Dataset/jbe2-ek4r).  

  

In [16]:
ipywidgets.VBox([mySelectedLabel1a, mySelectedLabel1b, ipywidgets.HBox([fig_sect, fig_subsect])])

### 2. Comparison with national averages
How does Washington state's greenhouse gas profile compare with the US average? Differences in the structure of state and national datasets make precise comparison difficult. But one stark point of contrast leaps out: whereas in Washington state the carbon footprint of power plants is less than a third of that of the transport industry, nationally the two sectors run head-to-head as the biggest greenhouse gas emitters. This demonstrates the sizable scale of emissions reduction made possible by switching to green power generation. 

Data is provided by the United States Environmental Protection Agency (EPA) (available at:
https://cfpub.epa.gov/ghgdata/inventoryexplorer/)

In [17]:
ipywidgets.VBox([mySelectedLabelEa, mySelectedLabelEb, fig_epa])

### 3. Washington state: regional variation 

How consistent are industry practices across the state? Which regions and sectors should be targeted for further carbon footprint reductions? This heatmap and accompanying bar chart reveal considerable regional variation. If we look at power plants, we can see from the heatmap that Lewis county has the highest emissions. The interactive bar chart reveals this county is home to state's last surviving coal plant. Checking again the breakdown of power plant emissions statewide (first visualization), we see this single coal plant produces higher emissions than all other fossil-fuel power plants combined.

Data is again from the Washington Department of Ecology dataset, cited above.

In [18]:
ipywidgets.VBox([mySelectedLabel2, ipywidgets.HBox([fig_hm, fig_bar4])])

![CountyMap](WashingtonMap.gif "CountyMap")

## Acknowledgements
These visualizations adapt design features and a substantial amount of code from class examples and from the solution to the Assignment 6 problem provided by Dr. Jill Naiman ("hwex.ipynb", personal communication, 3/15/2019). I also thank Dr. Naiman for helping with numerous individual problems, and 590 DVO classmates for their valuable suggestions.

Washington county map: http://www.dva.wa.gov/benefits/county-map (accessed 4/23/19)

Code for embedding git file adapted from: https://stackoverflow.com/questions/51527868/how-do-i-embed-a-gif-in-jupyter-notebook (accessed 4/23/19)